In [2]:
from dbrepo.RestClient import RestClient
from typing import List
import pandas as pd

# DBRepo

In [3]:
client = RestClient(endpoint="http://test.dbrepo.tuwien.ac.at", username="daria.stefan",
                    password="F@ntasiaRed7712") #so secure so safe much wow

user = client.get_user(user_id="fe88dee0-b5fa-103f-925a-771c9581a932")
print(f"user info: {user}")

user info: id='fe88dee0-b5fa-103f-925a-771c9581a932' username='daria.stefan' attributes=UserAttributes(theme='light', language='en', orcid='https://orcid.org/0009-0008-0586-9350', affiliation='0000 0004 1937 0669') qualified_name='Daria Stefan — @daria.stefan' given_name='Daria' family_name='Stefan' name='Daria Stefan'


In [4]:
databases = client.get_databases()

# Loop through and print name + id of each database
for db in databases:
    print(f"Name: {db.name}, ID: {db.id}")

Name: NFL Stadium Attendance, ID: fcedf17b-1ff2-4730-a314-af2d720d2c3f
Name: customer_personality_analysis, ID: f5fb6f43-4589-4c69-913a-23ee03f768b6
Name: Py_Illicit_Trade_KG, ID: f5ede80d-828b-47a0-bdf7-87fdffff815a
Name: ML and semi-automatic KG enrichment, ID: f364ce6f-6a52-43d5-bcd7-bc98e709c72c
Name: loan-dataset, ID: f234cb20-706f-4995-9ab9-89bcbf25d027
Name: Online Food Ordering System, ID: ef19ab12-0720-485c-b133-57a618a55bb8
Name: Py_Illicit_Trade_KG, ID: e7eab916-a14f-44d6-8457-ecc542248aa5
Name: Py_Illicit_Trade_KG, ID: e30f2ca5-ba60-4587-849b-4a496abfc186
Name: Airbnb Listings US 2023/12331410, ID: cf2d0d37-7b08-4b2b-841c-90517e95937a
Name: Py_Illicit_Trade_KG, ID: c834a548-f963-468b-9c4a-5b5f81b28c9a
Name: synthetic_student_data, ID: c45328ec-80f7-497d-8be6-c3505489ddb4
Name: Rainfall Prediction, ID: b71909fb-f916-4255-87cb-31dea8175ee0
Name: IoMT_dataset, ID: b3ac4b9e-f066-48f9-8bdd-326500cb1849
Name: Py_Illicit_Trade_KG, ID: b3616b9c-9ef6-4cde-b64e-c03f899c1225
Name: owi

In [194]:
database_id = "e30f2ca5-ba60-4587-849b-4a496abfc186"

In [161]:
data_df = pd.read_csv('data1.csv')  # The original triples with IDs
data_df = data_df.set_index('ID')

In [195]:
data_df

,head,relationship,tail
ID,,,
1,Abraham Trust,partnered,Geoffrey Jenkinson
2,Benjamin Bishop Johnson,appraised_art_for,David Swetnam
3,Donald Hales,appraised_art_for,Leonardo Patterson
4,Giacomo Medici,appraised_art_for,Nikolas Koutoulakis
5,Jack Ogden,appraised_art_for,Michael Ward Gallery
...,...,...,...
1200,Robert Hecht,sold_antiquities_to,Marion True
1201,Giacomo Medici,sold_antiquities_to,Marion True
1202,Robert Hecht,implicated_with,Marion True


In [196]:
##uploat table
client.create_table(database_id, "illicit antiques trade network", True, True, data_df,
                     "triples describing relationships between people, institutions, countries and the movement of antiques and artifacts", True)

TableBrief(id='7dfe8ed0-eaad-41e9-9987-55ee027ee983', database_id='e30f2ca5-ba60-4587-849b-4a496abfc186', name='illicit antiques trade network', description='triples describing relationships between people, institutions, countries and the movement of antiques and artifacts', internal_name='illicit_antiques_trade_network', is_versioned=True, is_public=True, is_schema_public=True, owned_by='fe88dee0-b5fa-103f-925a-771c9581a932')

In [197]:
internal_name='illicit_antiques_trade_network'

In [198]:
def get_all_tables(client, database_id: str) -> List:
    """
    List all tables in a given database by ID.
    
    :param client: Authenticated RestClient
    :param database_id: The database UUID
    :return: List of (table name, table id)
    """
    url = f'/api/database/{database_id}/table'
    response = client._wrapper(method="get", url=url)
    
    if response.status_code == 200:
        tables_json = response.json()
        # you may need to validate this using TypeAdapter if TableBrief or similar exists
        return [(table["name"], table["id"]) for table in tables_json]

    raise Exception(f"Failed to get tables: {response.status_code}: {response.text}")



In [199]:
tables = get_all_tables(client, database_id)

# Print them out
for name, table_id in tables:
    print(f"Table Name: {name}, Table ID: {table_id}")

Table Name: illicit antiques trade network, Table ID: 7dfe8ed0-eaad-41e9-9987-55ee027ee983


In [212]:
table_id="7dfe8ed0-eaad-41e9-9987-55ee027ee983"
table_name = "illicit_antiques_trade_network"

In [202]:
#get table
def fetch_all_table_data(client, database_id: str, table_id: str):
    """
    Fetch all data from a table using pagination and return it as a pandas DataFrame.

    :param client: The API client.
    :param database_id: The database ID.
    :param table_id: The table ID.
    :return: A pandas DataFrame containing all the table data.
    """
    all_data = []
    page = 0
    size = 100  # You can adjust the page size if needed
    
    while True:
        table_data = client.get_table_data(database_id, table_id, page, size)
        
        if table_data.empty:
            break  # No more data
        
        all_data.append(table_data)
        page += 1  # Go to the next page

    # Concatenate all pages into one big DataFrame
    full_data = pd.concat(all_data, ignore_index=True)
    
    return full_data


In [203]:
full_data

,head,relationship,tail,id
0,Abraham Trust,partnered,Geoffrey Jenkinson,1
1,Benjamin Bishop Johnson,appraised_art_for,David Swetnam,2
2,Donald Hales,appraised_art_for,Leonardo Patterson,3
3,Giacomo Medici,appraised_art_for,Nikolas Koutoulakis,4
4,Jack Ogden,appraised_art_for,Michael Ward Gallery,5
...,...,...,...,...
1199,Robert Hecht,sold_antiquities_to,Marion True,1200
1200,Giacomo Medici,sold_antiquities_to,Marion True,1201
1201,Robert Hecht,implicated_with,Marion True,1202
1202,Giacomo Medici,implicated_with,Marion True,1203


## Create query to split into subsets according to how the data was split

In [204]:
df = split_df.rename(columns={"Unnamed: 0": "ID"})

In [206]:
# --- Extract ID lists per split ---
train_ids    = df.loc[df["split"] == "train",     "ID"].astype(str).tolist()
validate_ids = df.loc[df["split"] == "validation","ID"].astype(str).tolist()
test_ids     = df.loc[df["split"] == "test",      "ID"].astype(str).tolist()

# --- Step 3: Create regex strings ---
train_regex    = "^(" + "|".join(train_ids)    + ")$"
validate_regex = "^(" + "|".join(validate_ids) + ")$"
test_regex     = "^(" + "|".join(test_ids)     + ")$"

# Print the regex strings
print("Train IDs regex:   ", train_regex)
print("Validate IDs regex:", validate_regex)
print("Test IDs regex:    ", test_regex)


Train IDs regex:    ^(0|1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|28|29|30|31|32|33|34|35|36|37|38|39|40|41|42|43|44|45|46|47|48|49|50|51|52|53|54|55|56|57|58|59|60|61|62|63|64|65|66|67|68|69|70|71|72|73|74|75|76|77|78|79|80|81|82|83|84|85|86|87|88|89|90|91|92|93|94|95|96|97|98|99|100|101|102|103|104|105|106|107|108|109|110|111|112|113|114|115|116|117|118|119|120|121|122|123|124|125|126|127|128|129|130|131|132|133|134|135|136|137|138|139|140|141|142|143|144|145|146|147|148|149|150|151|152|153|154|155|156|157|158|159|160|161|162|163|164|165|166|167|168|169|170|171|172|173|174|175|176|177|178|179|180|181|182|183|184|185|186|187|188|189|190|191|192|193|194|195|196|197|198|199|200|201|202|203|204|205|206|207|208|209|210|211|212|213|214|215|216|217|218|219|220|221|222|223|224|225|226|227|228|229|230|231|232|233|234|235|236|237|238|239|240|241|242|243|244|245|246|247|248|249|250|251|252|253|254|255|256|257|258|259|260|261|262|263|264|265|266|267|268|269|270|271|

In [207]:
train_filter = FilterDefinition(
    type=FilterType.WHERE,
    column="id",        # Column name
    operator="REGEXP",      # 'in' operator for checking membership
    value=train_regex  # Join IDs into a comma-separated string
)

In [208]:
validation_filter = FilterDefinition(
    type=FilterType.WHERE,
    column="id",        # Column name
    operator="REGEXP",      # 'in' operator for checking membership
    value=validate_regex  # Join IDs into a comma-separated string
)

In [209]:
test_filter = FilterDefinition(
    type=FilterType.WHERE,
    column="id",        # Column name
    operator="REGEXP",      # 'in' operator for checking membership
    value=test_regex  # Join IDs into a comma-separated string
)

In [235]:
test_query = QueryDefinition(
    table="illicit_antiques_trade_network",  # Table name,  # Table name
    columns=["head", "relationship", "tail", "id"],  # Select all columns
    filter = [test_filter]
)


# Initialize variables for pagination
page = 0
size = 4000  # You can adjust this to a higher value to reduce the number of pages
all_data = []

# Call create_subset with the required parameters
timestamp = datetime.datetime.now()  # Optional, set to None to query current data


    # Call the create_subset method with the current page and size
result = client.create_subset(
        database_id=database_id,
        query=test_query,
        page=page,
        size=size,
        timestamp=timestamp
    )
    
   
# Now, final_data contains all the records
print(result)

2025-04-28 17:02:42,911 root         INFO   Created subset with id: 304d946a-2440-11f0-bf18-c60333c11641
                    head         relationship                   tail    id
0                Cancuén             base_for        Sylvanus Morley  1084
1                 Xultún             base_for        Sylvanus Morley  1085
2                 London             base_for         Vladimir Mefty  1086
3            Switzerland    transit_point_for       Morgantina Venus  1087
4          United States    transit_point_for  El Zotz Wooden Lintel  1088
..                   ...                  ...                    ...   ...
115  Lawrence Fleischman      implicated_with            Marion True  1199
116         Robert Hecht  sold_antiquities_to            Marion True  1200
117       Giacomo Medici  sold_antiquities_to            Marion True  1201
118         Robert Hecht      implicated_with            Marion True  1202
119       Giacomo Medici      implicated_with            Marion True  

In [226]:
client.get_subset_data(database_id, "304d946a-2440-11f0-bf18-c60333c11641",  0, 10)

,id,head,relationship,tail
0,1084,Cancuén,base_for,Sylvanus Morley
1,1085,Xultún,base_for,Sylvanus Morley
2,1086,London,base_for,Vladimir Mefty
3,1087,Switzerland,transit_point_for,Morgantina Venus
4,1088,United States,transit_point_for,El Zotz Wooden Lintel
5,1089,Switzerland,transit_point_for,Euphronios ( Sarpedon ) krater
6,1090,United States,transit_point_for,red-on-buff decorated pottery
7,1091,Japan,transit_point_for,El Salvadore
8,1092,Saudi Arabia,transit_point_for,El Salvadore
9,1093,United States,transit_point_for,El Salvadore


In [229]:
client.update_subset(database_id,"304d946a-2440-11f0-bf18-c60333c11641",True)

ValidationError: 3 validation errors for Query
type
  Input should be 'view' or 'query' [type=enum, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/enum
query_normalized
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
identifiers
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [232]:
validation_query = QueryDefinition(
    table="illicit_antiques_trade_network",  # Table name,  # Table name
    columns=["head", "relationship", "tail", "id"],  # Select all columns
    filter = [validation_filter]
)


# Initialize variables for pagination
page = 0
size = 4000  # You can adjust this to a higher value to reduce the number of pages
all_data = []

# Call create_subset with the required parameters
timestamp = datetime.datetime.now()  # Optional, set to None to query current data


    # Call the create_subset method with the current page and size
result = client.create_subset(
        database_id=database_id,
        query=validation_query,
        page=page,
        size=size,
        timestamp=timestamp
    )
    
   
# Now, final_data contains all the records
print(result)

2025-04-28 17:01:30,228 root         INFO   Created subset with id: aa776d4d-2441-11f0-8632-66164a21a937
                    head   relationship                           tail    id
0                 London  auctioned_for                    Vaman Ghiya   963
1               New York  auctioned_for                    Vaman Ghiya   964
2              Sotheby's  auctioned_for                    Vaman Ghiya   965
3          Mat Securitas    belonged_to                 Giacomo Medici   966
4       Hôtel des Ventes  auctioned_for                    Guy Loudmer   967
..                   ...            ...                            ...   ...
116              Finland       base_for              Larisa Zavadskaya  1079
117  Museum of Fine Arts      possessed                   Maya pottery  1080
118                Perth       base_for                    Ong Soo Hin  1081
119             New York       base_for            Phoenix Ancient Art  1082
120        South America       base_for  Roger C

In [233]:
client.get_subset_data(database_id, "aa776d4d-2441-11f0-8632-66164a21a937",  0, 10)


,id,head,relationship,tail
0,963,London,auctioned_for,Vaman Ghiya
1,964,New York,auctioned_for,Vaman Ghiya
2,965,Sotheby's,auctioned_for,Vaman Ghiya
3,966,Mat Securitas,belonged_to,Giacomo Medici
4,967,Hôtel des Ventes,auctioned_for,Guy Loudmer
5,968,Hôtel des Ventes,auctioned_for,Hervé Poulin
6,969,eBay,auctioned_for,Hoi An ceramics
7,970,Arts of Asia,auctioned_for,John Eskenazi
8,971,Bonhams,auctioned_for,Joseph Garnish
9,972,Bonhams,auctioned_for,Marquis of Tavistock


In [234]:
client.update_subset(database_id,"aa776d4d-2441-11f0-8632-66164a21a937",True)

ValidationError: 3 validation errors for Query
type
  Input should be 'view' or 'query' [type=enum, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/enum
query_normalized
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
identifiers
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [237]:
train_query = QueryDefinition(
    table="illicit_antiques_trade_network",  # Table name
    columns=["head", "relationship", "tail", "id"],  # Select all columns
    filter = [train_filter]
)


# Initialize variables for pagination
page = 0
size = 4000  # You can adjust this to a higher value to reduce the number of pages
all_data = []

# Call create_subset with the required parameters
timestamp = datetime.datetime.now()  # Optional, set to None to query current data


    # Call the create_subset method with the current page and size
result = client.create_subset(
        database_id=database_id,
        query=train_query,
        page=page,
        size=size,
        timestamp=timestamp
    )
    
   
# Now, final_data contains all the records
print(result)

2025-04-28 17:03:31,126 root         INFO   Created subset with id: f250b964-2441-11f0-bf18-c60333c11641
                        head       relationship                  tail   id
0              Abraham Trust          partnered    Geoffrey Jenkinson    1
1    Benjamin Bishop Johnson  appraised_art_for         David Swetnam    2
2               Donald Hales  appraised_art_for    Leonardo Patterson    3
3             Giacomo Medici  appraised_art_for   Nikolas Koutoulakis    4
4                 Jack Ogden  appraised_art_for  Michael Ward Gallery    5
..                       ...                ...                   ...  ...
957                     Asia         source_for          Robert Olson  958
958                 Thailand         source_for          Robert Olson  959
959                   Hoi An           base_for         Captain Trang  960
960               Christie's      auctioned_for           Barry Stern  961
961               Christie's      auctioned_for           Vaman Ghiya 

In [238]:
client.get_subset_data(database_id, "f250b964-2441-11f0-bf18-c60333c11641",  0, 10)

,id,head,relationship,tail
0,1,Abraham Trust,partnered,Geoffrey Jenkinson
1,2,Benjamin Bishop Johnson,appraised_art_for,David Swetnam
2,3,Donald Hales,appraised_art_for,Leonardo Patterson
3,4,Giacomo Medici,appraised_art_for,Nikolas Koutoulakis
4,5,Jack Ogden,appraised_art_for,Michael Ward Gallery
5,6,Joel Malter,appraised_art_for,Robert Olson
6,7,John Betts,appraised_art_for,Michael Ward Gallery
7,8,Paul Clifford,appraised_art_for,Leonardo Patterson
8,9,Roxanna Brown,appraised_art_for,Jonathan and Cari Markell
9,10,Edward Johnson,caught_by,Immigration and Customs Enforcement


In [239]:
client.update_subset(database_id,"f250b964-2441-11f0-bf18-c60333c11641",True)

ValidationError: 3 validation errors for Query
type
  Input should be 'view' or 'query' [type=enum, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/enum
query_normalized
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
identifiers
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [240]:
get_all_subsets(client, database_id)

[('304d946a-2440-11f0-bf18-c60333c11641', '<no-title>'),
 ('aa776d4d-2441-11f0-8632-66164a21a937', '<no-title>'),
 ('f250b964-2441-11f0-bf18-c60333c11641', '<no-title>')]

## Identification - fails and is not documented properly :(

In [241]:
from dbrepo.api.dto import License
# Create a License instance
license = License(
    identifier="CC-BY-4.0",  # License identifier, for example, the license code
    uri="https://creativecommons.org/licenses/by/4.0/",  # URI pointing to the license details
    description="This is a Creative Commons Attribution 4.0 International License"  # Description of the license
)


In [246]:
from dbrepo.RestClient import RestClient
from dbrepo.api.dto import IdentifierType, CreateIdentifierFunder, CreateIdentifierDescription, \
    CreateIdentifierTitle, Identifier, CreateIdentifierCreator, CreateRelatedIdentifier, RelatedIdentifierType, \
    RelatedIdentifierRelation


client.create_identifier(
    database_id=database_id, 
    type = IdentifierType.DATABASE, 
    titles=[CreateIdentifierTitle(title="illicit_antiques_trade_network")],
    publisher="TU Wien",
    creators=[CreateIdentifierCreator(
        creator_name="Daria Stefan",
        name_identifier="https://orcid.org/0009-0008-0586-9350")],
    publication_year=2025,
  )

print(f"identifier id: {identifier.id}")

ValidationError: 1 validation error for CreateIdentifier
descriptions
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [249]:
from dbrepo.RestClient import RestClient
from dbrepo.api.dto import IdentifierType, CreateIdentifierFunder, CreateIdentifierDescription, \
    CreateIdentifierTitle, Identifier, CreateIdentifierCreator, CreateRelatedIdentifier, RelatedIdentifierType, \
    RelatedIdentifierRelation


client.create_identifier(
    database_id="e30f2ca5-ba60-4587-849b-4a496abfc186", 
    type = IdentifierType.DATABASE, 
    titles=[CreateIdentifierTitle(title="illicit antiques trade network")],
    publisher="TU Wien",
    creators=[CreateIdentifierCreator(
        creator_name="Daria Stefan",
        name_identifier="https://orcid.org/0009-0008-0586-9350")],
    publication_year=2025,
  )

print(f"identifier id: {identifier.id}")

ValidationError: 1 validation error for CreateIdentifier
descriptions
  Input should be a valid list [type=list_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

## utils

In [140]:
#get subsets
def get_all_subsets(client, database_id: str):
    """
    List all subsets in a given database.
    Returns a list of (subset_id, name) tuples.
    """
    url = f'/api/database/{database_id}/subset'
    response = client._wrapper(method="get", url=url)

    if response.status_code == 200:
        subsets = response.json()
        results = []
        for subset in subsets:
            subset_id = subset['id']
            try:
                name = subset['identifiers'][0]['titles'][0]['title']
            except (KeyError, IndexError, TypeError):
                name = "<no-title>"
            results.append((subset_id, name))
        return results

    raise Exception(f"Failed to get subsets: {response.status_code}: {response.text}")




In [141]:
get_all_subsets(client, database_id)

[('02f01bb4-2433-11f0-8632-66164a21a937', '<no-title>'),
 ('11a9f85b-2411-11f0-8632-66164a21a937', '<no-title>'),
 ('12561777-243a-11f0-8632-66164a21a937', '<no-title>'),
 ('14014c45-21c9-11f0-8d13-0e8cdc00dc05', 'Test Set'),
 ('19f1ebd4-21c5-11f0-bed7-02bbcde59723', '<no-title>'),
 ('1c379f05-243a-11f0-8632-66164a21a937', '<no-title>'),
 ('2239439b-243a-11f0-8632-66164a21a937', '<no-title>'),
 ('2475f958-21c4-11f0-8d13-0e8cdc00dc05', 'Training Set'),
 ('38ba5311-21c5-11f0-8d13-0e8cdc00dc05', 'Validation Set'),
 ('3c5e787f-2434-11f0-8632-66164a21a937', '<no-title>'),
 ('47acef31-2439-11f0-8632-66164a21a937', '<no-title>'),
 ('4dceff09-2439-11f0-97fd-36086aeef04e', '<no-title>'),
 ('57b9bbd0-243a-11f0-bf18-c60333c11641', '<no-title>'),
 ('65e04dc5-2439-11f0-bf18-c60333c11641', '<no-title>'),
 ('6d2a3ed6-243a-11f0-8632-66164a21a937', '<no-title>'),
 ('6f3725ba-2439-11f0-8632-66164a21a937', '<no-title>'),
 ('7711a19f-2439-11f0-8632-66164a21a937', '<no-title>'),
 ('7d10c992-2439-11f0-8632-

In [ ]:
def download_subset_data(client, database_id: str, subset_id: str = None, title: str = None):
    """
    Download data from a specific subset based on subset_id or title and save it as a CSV.
    If subset_id is provided, it will fetch data using that ID. 
    If title is provided, it will find the subset with the matching title and fetch its data.
    """
    if not subset_id and not title:
        raise ValueError("Either subset_id or title must be provided.")
    
    if title:
        # Search for the subset by title
        all_subsets = get_all_subsets(client, database_id)  # This should return a list of tuples (subset_id, title)
        subset = next((s for s in all_subsets if s[1] == title), None)
        if subset:
            subset_id = subset[0]
        else:
            raise ValueError(f"No subset found with title: {title}")

    all_data = []
    page = 0
    size = 100  # Adjust the page size as needed
    
    while True:
        subset_data = client.get_subset_data(database_id, subset_id, page, size)
        if subset_data.empty:
            break  # Exit the loop when no more data is found
        
        all_data.append(subset_data)
        page += 1  # Move to the next page
    
    # Concatenate all pages into one DataFrame
    full_data = pd.concat(all_data, ignore_index=True)
    
    # Save the full data as a CSV
    file_name = f"{title.replace(' ', '_') if title else subset_id}_data.csv"
    full_data.to_csv(file_name, index=False)
    print(f"Data for subset ID '{subset_id}' or title '{title}' saved as {file_name}")


# TUWRD

In [1]:
import requests
url = "https://researchdata.tuwien.ac.at/api/records"
headers = {
    "Authorization": "Bearer {your-access-token}",
    "Content-Type": "application/json"
}
data = {
    "metadata": {
        "creators": [
            {
                "person_or_org": {
                    "family_name": "Doe",
                    "given_name": "Jane",
                    "type": "personal",
                }
            }
        ],
        "publication_date": "2025-01-09",
        "publisher": "TU Wien",
        "resource_type": {"id": "dataset"},
        "title": "New Dataset Title",
    },
    "files": {"enabled": True},
}

response = requests.post(url, json=data, headers=headers)
print(response.json())

{'status': 403, 'message': 'Permission denied.'}


In [ ]:
import requests
url = "https://researchdata.tuwien.ac.at/api/records/abcde-12345/draft/pids/doi"
headers = {
    "Authorization": "Bearer {your-access-token}",
}

response = requests.post(url, headers=headers)
print(response.json())